# **Detectar arritmias cardíacas mediante señales de ECG parcialmente etiquetadas**
### INF395 Introducción a las Redes Neuronales and Deep Learning
- Estudiante: Alessandro Bruno Cintolesi Rodríguez
- ROL: 202173541-0

## 1. Importación de Librerías
Importamos todas las librerías necesarias, incluyendo PyTorch, Sklearn, Pandas y Matplotlib.

In [ ]:
# === General / Utilidad ===
import os
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# === PyTorch / PyTorch Lightning ===
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

# === Scikit-learn ===
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
	f1_score,
	accuracy_score,
	confusion_matrix,
)

from itertools import cycle
from tqdm import tqdm

## 2. Configuración Global y Hiperparámetros
Definimos las variables globales, hiperparámetros y seteamos el dispositivo (GPU o CPU) que se usará para el entrenamiento.

In [ ]:
# Seteamos la semilla
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
pl.seed_everything(SEED, workers=True)

# Seteamos el dispositivo
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True
print("Using device:", DEVICE)
if DEVICE.type == "cuda":
	print("GPU:", torch.cuda.get_device_name(0))

In [ ]:
TEST_SIZE = 0.2              # Reservamos el 20% de los datos etiquetados para validar y evitar sobreajuste.
BATCH_SIZE = 32              # El modelo procesa 32 ECGs simultáneamente antes de actualizar sus pesos.
EPOCHS = 50                  # Cantidad de veces que el modelo verá el set de datos completo durante el entrenamiento.
LEARNING_RATE = 1e-3         # Velocidad de aprendizaje; controla qué tan rápido se ajustan los pesos del modelo.
CONFIDENCE_THRESHOLD = 0.95  # (FixMatch) Solo confiamos en las predicciones de datos sin etiqueta si la certeza > 95%.
LAMBDA_U = 1.0               # Factor de equilibrio: da igual importancia a la pérdida supervisada y a la no supervisada.
NUM_CLASSES = 5              # Total de categorías de arritmias (0-4) que el modelo debe aprender a clasificar.

## 3. Carga y Preprocesamiento de Datos
Realizamos los siguientes pasos:
1.  **Cargar** los archivos `train_semi_supervised.csv` y `test_semi_supervised.csv`.
2.  **Separar** el set de entrenamiento en datos **etiquetados** y **no etiquetados** (basado en `NaN`).
3.  **Crear un set de Validación** (con `TEST_SIZE`) a partir de los datos *etiquetados*, usando una división estratificada para mantener el balance.
4.  **Normalizar** los datos: ajustamos un `StandardScaler` (Z-Score) *solo* con `X_train_labeled` y luego transformamos todos los sets (`X_unlabeled`, `X_val`, `X_test`).
5.  **Reformatear** los datos a la forma `(N, 1, 187)` requerida por la 1D-CNN de PyTorch.

In [ ]:
# --- 1. Carga de Datos ---
try:
	# Cargamos el set de entrenamiento: aquí están mezclados los datos con etiqueta y los sin etiqueta (NaN).
	train_df = pd.read_csv("ecg_signals/train_semi_supervised.csv")

	# Cargamos el set de prueba: datos "futuros" que el modelo nunca verá durante el entrenamiento.
	test_df = pd.read_csv("ecg_signals/test_semi_supervised.csv")

except FileNotFoundError:
	# Bloque de seguridad: detiene la ejecución limpiamente si la ruta de los archivos es incorrecta.
	print("Error: No se encontraron los archivos CSV. Verifica la ruta.")
	exit()

In [ ]:
print(f"Forma (shape) original de entrenamiento: {train_df.shape}")
print(f"Forma (shape) original de testing: {test_df.shape}")

In [ ]:
# --- 2. Procesamiento del Set de Test ---

# Extraemos la señal ECG (features): ignoramos la col 0 (ID) y tomamos de la 1 a la 187.
X_test_raw = test_df.iloc[:, 1:188].values

# Extraemos la etiqueta (target): columna 188, y forzamos que sean enteros para el modelo.
y_test_raw = test_df.iloc[:, 188].values.astype(int)

In [ ]:
# --- 3. Procesamiento del Set de Entrenamiento (SSL) ---

# Creamos una "máscara" lógica: True si la fila tiene etiqueta, False si es NaN (sin etiqueta).
labeled_mask = train_df.iloc[:, 187].notna()

# Separamos los datos en dos mundos distintos:
# 1. labeled_df: Datos que el médico etiquetó. Usaremos esto para aprendizaje Supervisado normal.
labeled_df = train_df[labeled_mask]

# 2. unlabeled_df: Datos sin etiqueta (la mayoría). Usaremos esto con FixMatch para aprender la estructura de la señal.
unlabeled_df = train_df[~labeled_mask]

In [ ]:
print(f"\nDatos etiquetados encontrados: {len(labeled_df)}")
print(f"Datos NO etiquetados encontrados: {len(unlabeled_df)}")

In [ ]:
# --- 4. Crear X/y para Labeled y Unlabeled ---

# Extraemos la matriz de características (señales ECG) de los datos etiquetados.
X_labeled_full = labeled_df.iloc[:, 0:187].values

# Extraemos el vector de etiquetas (diagnósticos) correspondientes. 
# Es vital usar .astype(int) para que la función de pérdida de PyTorch lo acepte.
y_labeled_full = labeled_df.iloc[:, 187].values.astype(int)

# Extraemos solo las características de los datos NO etiquetados.
# Nota: Aquí no extraemos 'y' porque no existe (es lo que el modelo intentará adivinar).
X_unlabeled_raw = unlabeled_df.iloc[:, 0:187].values

In [ ]:
# --- 5. Crear Set de Validación (Estratificado) ---

# Subdividimos los datos etiquetados: 
# 'X_train_labeled_raw': Para entrenar al modelo.
# 'X_val_raw': Para medir su rendimiento en datos no vistos (Validación).
X_train_labeled_raw, X_val_raw, y_train_labeled, y_val = train_test_split(
	X_labeled_full,
	y_labeled_full,
	test_size=TEST_SIZE,      # Reservamos el 20% (definido arriba) para validación.
	stratify=y_labeled_full,  # Mantiene la misma proporción de clases (desbalance) en train y val.
	random_state=SEED         # Semilla fija para que la división sea reproducible siempre igual.
)

In [ ]:
print("\n--- División Final (Etiquetados) ---")
print(f"Muestras de entrenamiento (etiquetadas): {len(X_train_labeled_raw)}")
print(f"Muestras de validación (etiquetadas): {len(X_val_raw)}")

In [ ]:
# --- 6. Normalización (StandardScaler) ---

# Inicializamos el escalador. Usaremos Z-Score (restar media, dividir por desviación estándar).
scaler = StandardScaler()

# Calculamos la media y desviación estándar SOLO con los datos de entrenamiento.
scaler.fit(X_train_labeled_raw)

# Ahora usamos esa calculadora calibrada para transformar TODOS los conjuntos.
# Esto asegura que todos los datos estén en la misma escala matemática.
X_train_labeled_scaled = scaler.transform(X_train_labeled_raw)
X_unlabeled_scaled = scaler.transform(X_unlabeled_raw)
X_val_scaled = scaler.transform(X_val_raw)
X_test_scaled = scaler.transform(X_test_raw)

print("\nDatos normalizados (escalados) exitosamente.")

In [ ]:
# --- 7. Reformatear (Reshape) para 1D-CNN ---

# Las capas Conv1d de PyTorch exigen una entrada de 3 dimensiones: (Batch, Canales, Tiempo).
# Nuestros datos actuales son 2D: (Muestras, 187).
# Usamos np.newaxis para insertar una dimensión de "Canal" (que es 1, ya que es un solo sensor ECG).
# La forma cambia de (N, 187) a (N, 1, 187).

X_train_labeled = X_train_labeled_scaled[:, np.newaxis, :]
X_unlabeled = X_unlabeled_scaled[:, np.newaxis, :]
X_val = X_val_scaled[:, np.newaxis, :]
X_test = X_test_scaled[:, np.newaxis, :]

# Las etiquetas (y) se mantienen igual, ya que son solo una lista de respuestas correctas.
y_test = y_test_raw

In [ ]:
print("\n--- Forma Final de los Datos (Listos para el Modelo) ---")
print(f"X_train_labeled: {X_train_labeled.shape}")
print(f"y_train_labeled: {y_train_labeled.shape}")
print(f"X_unlabeled: {X_unlabeled.shape}")
print(f"X_val: {X_val.shape}")
print(f"y_val: {y_val.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")

## 4. Análisis Exploratorio (Desbalance)
Verificamos el **fuerte desbalance de clases** en nuestros datos etiquetados. Esto confirma la necesidad de usar métricas como F1-Macro y una función de pérdida especializada como Focal Loss.

In [ ]:
print("\n--- Análisis de Desbalance de Clases (Datos Etiquetados) ---")

# Contar las clases en el set etiquetado completo
class_counts = pd.Series(y_labeled_full).value_counts().sort_index()
print(class_counts)

# Calcular porcentajes
class_percentages = (class_counts / len(y_labeled_full)) * 100
print("\nPorcentaje de cada clase:")
print(class_percentages)

## 5. Definición del Modelo y Función de Pérdida

### 5.1. Modelo 1D-CNN
Definimos la arquitectura de nuestra Red Convolucional 1D (`ECG_1D_CNN`), ideal para aprender patrones en series temporales como los ECG.

![Diagrama Arquitectura ECG](ecg_model.png)

In [ ]:
# Definimos la arquitectura del modelo
class ECG_1D_CNN(nn.Module):
	def __init__(self, num_classes=5):
		super(ECG_1D_CNN, self).__init__()
		
		# --- Extractor de Características ---
		
		# Bloque 1: Detecta patrones simples de bajo nivel.
		self.conv_block1 = nn.Sequential(
			# Conv1d: Escanea la señal. Entra 1 canal (el ECG), salen 64 mapas de características.
			nn.Conv1d(in_channels=1, out_channels=64, 
					  kernel_size=5, stride=1, padding=2),
			# BatchNorm: Normaliza los valores para que el entrenamiento sea estable y rápido.
			nn.BatchNorm1d(64),
			# ReLU: "Enciende" las neuronas si encuentra algo positivo, apaga si es negativo.
			nn.ReLU(),
			# MaxPool: Reduce la señal a la mitad (187 -> 93).
			nn.MaxPool1d(kernel_size=2, stride=2)
		)
		
		# Bloque 2: Detecta patrones intermedios combinando los anteriores.
		self.conv_block2 = nn.Sequential(
			# Aumentamos la profundidad a 128 canales para capturar más detalles.
			nn.Conv1d(in_channels=64, out_channels=128, 
					  kernel_size=5, stride=1, padding=2),
			# BatchNorm: Normaliza los valores para que el entrenamiento sea estable y rápido.
			nn.BatchNorm1d(128),
			# ReLU: "Enciende" las neuronas si encuentra algo positivo, apaga si es negativo.
			nn.ReLU(),
			# Reducimos la dimensión temporal otra vez (93 -> 46).
			nn.MaxPool1d(kernel_size=2, stride=2)
		)
		
		# Bloque 3: Detecta patrones complejos y abstractos de la arritmia.
		self.conv_block3 = nn.Sequential(
			# Máxima profundidad (256 canales).
			nn.Conv1d(in_channels=128, out_channels=256, 
					  kernel_size=5, stride=1, padding=2),
			# BatchNorm: Normaliza los valores para que el entrenamiento sea estable y rápido.
			nn.BatchNorm1d(256),
			# ReLU: "Enciende" las neuronas si encuentra algo positivo, apaga si es negativo.
			nn.ReLU(),
			# Última reducción (46 -> 23).
			nn.MaxPool1d(kernel_size=2, stride=2)
		)
		
		# --- Cabeza de Clasificación ---
		
		# Flatten: "Aplana" el cubo de características 3D a un vector largo 1D para poder clasificar.
		self.flatten = nn.Flatten() # Transforma (N, 256, 23) -> (N, 5888)
		
		# Classifier: Red neuronal densa (MLP) que toma la decisión final.
		self.classifier = nn.Sequential(
			nn.Linear(in_features=256 * 23, out_features=512),
			# ReLU: "Enciende" las neuronas si encuentra algo positivo, apaga si es negativo.
			nn.ReLU(),
			# Dropout: Apaga aleatoriamente el 50% de neuronas para evitar memorización (overfitting).
			nn.Dropout(0.5),
			# Capa final: Reduce todo a 5 números (las puntuaciones para cada clase).
			nn.Linear(in_features=512, out_features=num_classes)
		)

	def forward(self, x):
		"""
		Define el flujo de datos: desde la señal cruda hasta la predicción.
		"""
		# Pasamos la señal por los bloques extractores (convolucionales)
		x = self.conv_block1(x)
		x = self.conv_block2(x)
		x = self.conv_block3(x)
		
		# Preparamos los datos para la clasificación
		x = self.flatten(x)
		
		# Obtenemos los 'logits' (puntuaciones crudas) de las 5 clases
		logits = self.classifier(x)
		return logits

### 5.2. Focal Loss
Definimos la clase `FocalLoss`. Esta pérdida modificada nos ayudará a mitigar el desbalance de clases forzando al modelo a enfocarse en las muestras difíciles (clases minoritarias).

In [ ]:
class FocalLoss(nn.Module):
	def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
		super(FocalLoss, self).__init__()
		self.gamma = gamma  # El "foco": cuánto ignoramos los ejemplos fáciles (gamma=0 es CrossEntropy normal).
		self.alpha = alpha  # El "balance": pesos manuales para dar más importancia a clases minoritarias.
		self.reduction = reduction
		
		if self.alpha is not None:
			if not isinstance(self.alpha, torch.Tensor):
				self.alpha = torch.tensor(self.alpha)
				
	def forward(self, logits, targets):
		"""
		Calcula la pérdida Focal.
		Input: logits (Predicciones crudas del modelo), targets (Respuestas correctas)
		"""
		
		# 1. Log-Softmax: Convertimos los puntajes crudos en probabilidades logarítmicas.
		log_probs = F.log_softmax(logits, dim=1)
		
		# 2. Extraer la probabilidad de la clase correcta (p_t):
		# De las 5 probabilidades que da el modelo, 'gather' selecciona SOLO la que corresponde a la etiqueta real.
		log_pt_true = log_probs.gather(1, targets.view(-1, 1)).view(-1)
		
		# 3. Obtener la probabilidad real (pt):
		# Deshacemos el logaritmo para tener un valor entre 0 y 1.
		pt_true = log_pt_true.exp()
		
		# 4. Calcular el Factor de Modulación: (1 - pt)^gamma
		# - Si el modelo está muy seguro (pt -> 1), (1-pt) es casi 0 -> La pérdida se anula (lo ignoramos).
		# - Si el modelo se equivoca (pt -> 0), (1-pt) es casi 1 -> La pérdida se mantiene alta (aprendemos de esto).
		focal_term = (1 - pt_true)**self.gamma
		
		# 5. Pérdida Base (Cross Entropy):
		# Calculamos la entropía cruzada estándar (-log(pt)).
		ce_loss = -log_pt_true
		
		# 6. Combinación: Multiplicamos la pérdida estándar por nuestro factor de "foco".
		loss = focal_term * ce_loss
		
		# 7. Aplicar pesos Alpha (Opcional):
		# Si queremos forzar aún más el balance, multiplicamos por el peso específico de cada clase.
		if self.alpha is not None:
			if self.alpha.device != logits.device:
				self.alpha = self.alpha.to(logits.device)
			
			alpha_t = self.alpha.gather(0, targets)
			loss = alpha_t * loss
			
		# 8. Reducción: Devolvemos el promedio del error de todo el lote (batch).
		if self.reduction == 'mean':
			return loss.mean()
		elif self.reduction == 'sum':
			return loss.sum()
		else:
			return loss

## 6. Definición de Aumentaciones (FixMatch)
Definimos las dos funciones de aumentación requeridas por FixMatch:
* `aug_weak`: Añade ruido gaussiano leve.
* `aug_strong`: Añade ruido fuerte y "time masking" (secciones puestas a cero).

Luego, visualizamos un ejemplo para confirmar su efecto.

In [ ]:
# --- 1. AUMENTACIÓN DÉBIL ---

# Definimos la función de aumentación débil.
# Objetivo: Simular pequeñas variaciones naturales en la señal (como ruido del sensor o movimiento leve)
# para que el modelo sea robusto y no se "memorice" los valores exactos de cada píxel.
def aug_weak(x_batch, noise_level=0.01):
	"""
	Aplica una aumentación débil añadiendo ruido gaussiano.
	Input: x_batch (N, 1, 187)
	"""
	# Paso de seguridad: Creamos una copia independiente de los datos originales.
	x_aug = x_batch.clone()
	
	# Generamos el "ruido": una matriz de números aleatorios (distribución normal) con la misma forma que nuestros ECGs.
	# Multiplicamos por 'noise_level' (0.01) para mantener el ruido muy suave y no destruir la información clínica.
	noise = torch.randn_like(x_aug) * noise_level
	
	# Sumamos el ruido a la señal original.
	return x_aug + noise.to(x_batch.device)

In [ ]:
def aug_strong(x_batch, noise_level=0.1, num_masks=3, mask_size=20):
	"""
	Aplica aumentación fuerte: combina ruido intenso y borrado de secciones (Time Masking).
	Input: x_batch (N, 1, 187)
	"""
	# Paso de seguridad: Creamos una copia independiente de los datos originales.
	x_aug = x_batch.clone()
	
	# 1. Ruido Fuerte:
	# Añadimos 10 veces más ruido que en la versión débil (0.1 vs 0.01).
	noise = torch.randn_like(x_aug) * noise_level
	x_aug += noise.to(x_batch.device)
	
	# 2. Time Masking:
	# Vamos a "borrar" o poner a cero partes aleatorias del electrocardiograma.
	# Esto obliga al modelo a usar el contexto: debe adivinar la arritmia viendo solo fragmentos de la señal.
	N, C, L = x_aug.shape # N=Batch, C=1, L=187
	
	for i in range(N): # Procesamos cada paciente del lote uno por uno
		for _ in range(num_masks): # Aplicamos 3 cortes distintos por paciente
			# Elegimos un punto de inicio al azar, asegurándonos de no salirnos del final de la señal.
			t_start = torch.randint(0, L - mask_size, (1,)).item()
			
			# Ponemos a cero una ventana de 20 puntos.
			x_aug[i, :, t_start : t_start + mask_size] = 0.0
			
	return x_aug

In [ ]:
def plot_ecg_comparison(signals_list, titles_list):
	"""
	Grafica una lista de señales de ECG en subplots verticales para comparar.
	MUESTRA el gráfico en lugar de guardarlo.

	Args:
		signals_list (list): Lista de tensores de ECG.
		titles_list (list): Lista de strings para los títulos de cada subplot.
	"""
	if len(signals_list) != len(titles_list):
		print("Error: El número de señales no coincide con el número de títulos.")
		return

	num_signals = len(signals_list)

	fig, axes = plt.subplots(nrows=num_signals, ncols=1, 
								figsize=(15, 3 * num_signals), 
								sharex=True)

	if num_signals == 1:
		axes = [axes]

	for i in range(num_signals):
		ax = axes[i]
		
		signal_np = signals_list[i].cpu().detach().numpy().squeeze()
		
		ax.plot(signal_np)
		ax.set_title(titles_list[i], fontsize=14)
		ax.set_ylabel("Amplitud")
		ax.grid(True, linestyle='--', alpha=0.6)

	axes[-1].set_xlabel("Paso de Tiempo (Time Step)")

	plt.tight_layout()
	plt.show()

In [ ]:
# --- Ejemplo de uso ---
dummy_input = torch.randn(4, 1, 187).to(DEVICE)

weak_output = aug_weak(dummy_input)
strong_output = aug_strong(dummy_input)

sample_original = dummy_input[0]
sample_weak = weak_output[0]
sample_strong = strong_output[0]

plot_ecg_comparison(
	signals_list=[sample_original, sample_weak, sample_strong],
	titles_list=["1. ECG Original", "2. Aumentación Débil (Ruido)", "3. Aumentación Fuerte (Ruido + Masking)"]
)

## 7. Creación de Datasets y DataLoaders
Definimos las clases `LabeledECGDataset` y `UnlabeledECGDataset` para manejar nuestros datos. Luego, creamos los `DataLoader` para el entrenamiento (etiquetado y no etiquetado) y la validación/testeo.

In [ ]:
# --- 1. Definir los Datasets ---
class LabeledECGDataset(Dataset):
	"""Dataset para nuestros datos etiquetados (Entrenamiento Supervisado y Validación)"""
	def __init__(self, x_data, y_data):
		# Recibimos los datos (X) y las etiquetas (y).
		self.x_data = x_data 
		self.y_data = y_data
		
	def __len__(self):
		# Le dice a PyTorch cuántas muestras totales existen.
		return len(self.x_data)
	
	def __getitem__(self, idx):
		# Convertimos a Tensor cada vez que el modelo pide una muestra específica.
		x = torch.tensor(self.x_data[idx], dtype=torch.float32) # La señal
		y = torch.tensor(self.y_data[idx], dtype=torch.long)    # La etiqueta (0-4)
		return x, y

class UnlabeledECGDataset(Dataset):
	"""Dataset para datos NO etiquetados (Solo para FixMatch)"""
	def __init__(self, x_data):
		# Aquí solo recibimos X, porque no existen etiquetas (y).
		self.x_data = x_data
		
	def __len__(self):
		return len(self.x_data)
	
	def __getitem__(self, idx):
		# Convertimos a Tensor la señal individual.
		x = torch.tensor(self.x_data[idx], dtype=torch.float32)
		# Solo devolvemos X. El modelo tendrá que "imaginar" la etiqueta (pseudo-label).
		return x

In [ ]:
# --- 1. Crear Datasets y DataLoaders ---

# --- Datasets ---
# Envolvemos nuestras matrices numéricas en objetos 'Dataset'.
train_labeled_ds = LabeledECGDataset(X_train_labeled, y_train_labeled)
train_unlabeled_ds = UnlabeledECGDataset(X_unlabeled)
val_ds = LabeledECGDataset(X_val, y_val)
test_ds = LabeledECGDataset(X_test, y_test)

# --- DataLoaders ---
# Para ENTRENAMIENTO (labeled y unlabeled): shuffle=True
labeled_loader = DataLoader(train_labeled_ds, batch_size=BATCH_SIZE, shuffle=True)
unlabeled_loader = DataLoader(train_unlabeled_ds, batch_size=BATCH_SIZE, shuffle=True)

# Para VALIDACIÓN y TEST: shuffle=False
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

## 8. Funciones de Entrenamiento y Evaluación

### 8.1. train_fixmatch
Esta es la función principal que implementa la lógica de FixMatch en cada época:
1.  Calcula la pérdida supervisada (`loss_s`) en el batch etiquetado.
2.  Genera pseudo-etiquetas en el batch no etiquetado (con `aug_weak`).
3.  Filtra las pseudo-etiquetas usando el `CONFIDENCE_THRESHOLD`.
4.  Calcula la pérdida de consistencia (`loss_u`) usando `aug_strong`.
5.  Combina las pérdidas (`loss_s + LAMBDA_U * loss_u`) y retropropaga.

### 8.2. evaluate
Función estándar para evaluar el modelo en el set de validación o test. Devuelve el **F1-Macro Score** (nuestra métrica clave) y la matriz de confusión.

In [ ]:
def train_fixmatch(model, labeled_loader, unlabeled_loader, optimizer, loss_sup, loss_unsup, device, conf_threshold, lambda_u):
	"""
	Ejecuta una época de entrenamiento. Aquí es donde ocurre la lógica de FixMatch.
	"""
	model.train() # Activamos modo entrenamiento
	
	total_loss_s = 0.0
	total_loss_u = 0.0
	
	# Como tenemos muchos más datos NO etiquetados que etiquetados, 'cycle' reinicia el iterador 
	# de los no etiquetados infinitamente para que nunca nos quedemos sin ellos 
	# mientras recorremos una vez el set etiquetado.
	pbar = tqdm(zip(labeled_loader, cycle(unlabeled_loader)), total=len(labeled_loader))
	
	for (x_labeled, y_labeled), x_unlabeled in pbar:
		
		# Movemos todo a la GPU para velocidad
		x_labeled = x_labeled.to(device)
		y_labeled = y_labeled.to(device)
		x_unlabeled = x_unlabeled.to(device)
		
		# --- 1. Parte Supervisada (Lo clásico) ---
		# Entrenamos con los datos que SÍ tienen etiqueta.
		# Usamos aumentación débil para robustez básica.
		x_labeled_aug = aug_weak(x_labeled)
		logits_s = model(x_labeled_aug)
		loss_s = loss_sup(logits_s, y_labeled) 
		
		# --- 2. Parte No Supervisada (La magia de FixMatch) ---
		# Crear la Pseudo-Etiqueta (El "Maestro")
		# Usamos aumentación DÉBIL y NO calculamos gradientes.
		with torch.no_grad(): 
			x_unlabeled_w = aug_weak(x_unlabeled)
			logits_uw = model(x_unlabeled_w)
			
			# Calculamos la probabilidad de la predicción
			probs_uw = torch.softmax(logits_uw, dim=1)
			max_prob, pseudo_label = torch.max(probs_uw, dim=1)
			
			# Filtro de Confianza: Si el modelo duda (probabilidad < 95%),
			# creamos una máscara de 0 para ignorar este dato más adelante.
			mask = (max_prob >= conf_threshold).float()
			
		# Ahora tomamos la MISMA señal, le aplicamos aumentación FUERTE (difícil),
		# y forzamos al modelo a predecir lo mismo que predijo en la versión fácil.
		x_unlabeled_s = aug_strong(x_unlabeled)
		logits_us = model(x_unlabeled_s)
		
		# Calculamos el error entre la predicción difícil y la pseudo-etiqueta fácil.
		loss_u_all = loss_unsup(logits_us, pseudo_label)
		
		# Aplicamos la máscara: Solo aprendemos de los casos donde el modelo estaba seguro.
		loss_u = (loss_u_all * mask).mean()
		
		# --- 3. Optimización ---
		# Sumamos ambas pérdidas. Lambda_u controla el peso de la parte no supervisada.
		total_loss = loss_s + lambda_u * loss_u
		
		optimizer.zero_grad()   # Limpiar basura anterior
		total_loss.backward()   # Calcular gradientes
		optimizer.step()        # Actualizar pesos
		
		total_loss_s += loss_s.item()
		total_loss_u += loss_u.item()
		
		pbar.set_description(f"Loss_S: {loss_s.item():.4f} | Loss_U: {loss_u.item():.4f}")
		
	return total_loss_s / len(labeled_loader), total_loss_u / len(labeled_loader)

In [ ]:
def evaluate(model, val_loader, device):
	"""
	Evalúa el modelo en el set de validación.
	"""
	# --- 1. Modo Evaluación ---
	model.eval() 
	
	all_preds = []
	all_targets = []
	
	# --- 2. Optimización (Sin Gradientes) ---
	with torch.no_grad(): 
		for x_val, y_val in val_loader:
			x_val = x_val.to(device)
			y_val = y_val.to(device)
			
			logits = model(x_val)
			
			# --- 3. Decisión Final (Argmax) ---
			preds = torch.argmax(logits, dim=1)
			
			# Movemos los datos de la GPU a la CPU para que Scikit-learn pueda leerlos.
			all_preds.extend(preds.cpu().numpy())
			all_targets.extend(y_val.cpu().numpy())
			
	# --- 4. Métricas Clave ---
	# F1-Macro: Promedia el éxito de cada clase por separado. 
	f1_macro = f1_score(all_targets, all_preds, average='macro', zero_division=0)
	
	# Accuracy: El porcentaje total de aciertos.
	acc = accuracy_score(all_targets, all_preds)
	
	# Matriz de Confusión: Nos permite ver exactamente dónde se equivoca.
	cm = confusion_matrix(all_targets, all_preds)
	
	return acc, f1_macro, cm

In [ ]:
# --- 1. Inicializar Modelo, Pérdida y Optimizador ---
model = ECG_1D_CNN(num_classes=NUM_CLASSES).to(DEVICE)

# --- Definición de Funciones de Pérdida ---

# (Supervisado): Calcula el error promedio del batch.
# reduction='mean': Usado para los datos que TIENEN etiqueta real.
loss_sup = FocalLoss(gamma=2.0, alpha=None, reduction='mean').to(DEVICE)

# (No Supervisado): Calcula el error individual por muestra.
# reduction='none': Necesitamos el error individual para poder multiplicar por la máscara (0 o 1)
# y así anular el error de las muestras donde el modelo no estaba seguro.
loss_unsup = FocalLoss(gamma=2.0, alpha=None, reduction='none').to(DEVICE)

# El Optimizador (Adam): Recibe los parámetros del modelo y la tasa de aprendizaje (qué tan rápido debe hacer cambios).
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
# --- 2. Bucle de Entrenamiento ---

best_f1 = -1.0 
best_model_path = f"checkpoints/best_model_ecg.pth"

# Bucle Principal
for epoch in range(EPOCHS):
	print(f"\n--- Época {epoch+1}/{EPOCHS} ---")
	
	# 1. Fase de Aprendizaje (Train):
	# El modelo ve los datos, hace predicciones, calcula el error y ajusta sus pesos.
	avg_loss_s, avg_loss_u = train_fixmatch(
		model, 
		labeled_loader, 
		unlabeled_loader, 
		optimizer, 
		loss_sup, 
		loss_unsup, 
		DEVICE, 
		CONFIDENCE_THRESHOLD, 
		LAMBDA_U
	)
	
	print(f"Pérdida promedio (S): {avg_loss_s:.4f} | Pérdida promedio (U): {avg_loss_u:.4f}")
	
	# 2. Fase de Validación (Eval):
	# Evaluamos al modelo en datos que NO ha visto durante el entrenamiento.
	val_acc, val_f1, val_cm = evaluate(model, val_loader, DEVICE)
	print(f"Validación - Acc: {val_acc:.4f} | F1-Macro: {val_f1:.4f}")
	
	# 3. Checkpointing:
	# Solo guardamos el modelo en el disco si su F1-Macro es MEJOR que el mejor récord histórico.
	if val_f1 > best_f1:
		best_f1 = val_f1
		torch.save(model.state_dict(), best_model_path)
		print(f"🎉 Nuevo mejor modelo guardado ({best_model_path})")

## 9. Evaluación Final en Test Set
Cargamos el mejor modelo encontrado en el paso anterior y lo evaluamos contra el **conjunto de test** (que el modelo nunca ha visto) para obtener nuestras métricas de rendimiento finales.

In [ ]:
# --- 3. Evaluación Final (con el set de Test) ---
print("\n--- Entrenamiento Finalizado ---")
print("Cargando mejor modelo para evaluación final...")

# Cargamos el modelo 'best_model_path' que guardamos cuando obtuvo su mejor nota en validación.
model.load_state_dict(torch.load(best_model_path))

# Ejecutamos la evaluación sobre el Test Loader.
test_acc, test_f1, test_cm = evaluate(model, test_loader, DEVICE)

print(f"\n--- Resultados Finales (Test Set) ---")

# Reportamos Accuracy: % total de aciertos.
print(f"Accuracy: {test_acc:.4f}")

# Reportamos F1-Macro.
print(f"F1-Macro: {test_f1:.4f}")

print("Matriz de Confusión (Test):")
# Imprimimos la radiografía de los errores para analizar: ¿Qué clases está confundiendo entre sí?
print(test_cm)

## 10. Generación de Archivo de Submission
Finalmente, usamos el mejor modelo para generar las predicciones del archivo `test.csv` y las guardamos en el formato `submission.csv` requerido.

In [ ]:
# --- Generación del Archivo de Submission (Test) ---
print("\n--- Generando archivo CSV de submission ---")

# Cargar el mejor modelo guardado
model.load_state_dict(torch.load(best_model_path))

# --- 1. Cargar los IDs del CSV de test ---
try:
	test_df_original = pd.read_csv("ecg_signals/test_semi_supervised.csv")
	test_ids = test_df_original.iloc[:, 0].values
	print(f"IDs de test cargados: {len(test_ids)} encontrados.")
except FileNotFoundError:
	print("Error: No se pudo cargar 'test_semi_supervised.csv' para obtener los IDs.")
	exit()
	
# --- 2. Obtener todas las predicciones del modelo ---
model.eval()
all_predictions = []

with torch.no_grad():
	# Iteramos sobre el test_loader (que tiene shuffle=False)
	for x_batch, y_true_labels in test_loader:
		x_batch = x_batch.to(DEVICE)
		
		# Obtener logits
		logits = model(x_batch)
		
		# Obtener la predicción (la clase con mayor logit)
		preds = torch.argmax(logits, dim=1)
		
		# Guardar las predicciones
		all_predictions.extend(preds.cpu().numpy())

print(f"Predicciones generadas: {len(all_predictions)} hechas.")

# --- 3. Verificar y Crear el DataFrame ---
if len(test_ids) != len(all_predictions):
	print(f"¡Error! La cantidad de IDs ({len(test_ids)}) no coincide "
		  f"con la cantidad de predicciones ({len(all_predictions)}).")
	print("Verifica que el test_loader NO esté barajando (shuffle=False).")
else:
	# Crear el DataFrame con el formato solicitado
	submission_df = pd.DataFrame({
		'ID': test_ids,
		'label': all_predictions
	})
	
	# --- 4. Guardar en CSV ---
	output_filename = f"ecg_submittions/submission_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv"
	
	# index=False es crucial para que no añada una columna de índice
	submission_df.to_csv(output_filename, index=False)
	
	print(f"\n¡Éxito! Archivo de submission guardado en: {output_filename}")
	print("Vista previa de los resultados:")
	print(submission_df.head())

## 11. Búsqueda de Hiperparámetros (Grid Search)
Usamos `ParameterGrid` de Sklearn para definir una grilla de hiperparámetros a probar (Learning Rate, Threshold, Lambda, Gamma).

Iteramos sobre cada combinación, entrenamos un modelo desde cero y guardamos el `F1-Macro` de validación. Finalmente, seleccionamos la combinación con el mejor F1-Score.

In [ ]:
if __name__ == '__main__':
	# === 1. Setup Básico y Carga de Datos ===

	SEED = 42
	torch.manual_seed(SEED)
	np.random.seed(SEED)
	pl.seed_everything(SEED, workers=True)
	DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print("Using device:", DEVICE)

	# --- 1.1 Carga de Datos ---
	try:
		train_df = pd.read_csv("ecg_signals/train_semi_supervised.csv")
		test_df = pd.read_csv("ecg_signals/test_semi_supervised.csv")
	except FileNotFoundError:
		print("Error: No se encontraron los archivos CSV. Verifica la ruta.")
		exit()
	# --- 1.2 Procesamiento del Set de Test ---
	# Col 0: ID (lo ignoramos)
	# Col 1-187: Señal (Total 187 pasos)
	# Col 188: Clase
	X_test_raw = test_df.iloc[:, 1:188].values  # Columnas 1 a 187
	y_test_raw = test_df.iloc[:, 188].values.astype(int)

	# --- 1.3 Procesamiento del Set de Entrenamiento (SSL) ---
	# Col 0-186: Señal (Total 187 pasos)
	# Col 187: Clase
	labeled_mask = train_df.iloc[:, 187].notna()

	# Separar los DataFrames
	labeled_df = train_df[labeled_mask]
	unlabeled_df = train_df[~labeled_mask]
	
	# --- 1.4 Crear X/y para Labeled y Unlabeled ---
	# Datos Llenos Etiquetados (para crear train/val)
	X_labeled_full = labeled_df.iloc[:, 0:187].values
	y_labeled_full = labeled_df.iloc[:, 187].values.astype(int)

	# Datos No Etiquetados (solo X)
	X_unlabeled_raw = unlabeled_df.iloc[:, 0:187].values

	# --- 1.5 Crear Set de Validación (Estratificado) ---
	# Dividimos el set ETIQUETADO para crear un set de validación.
	X_train_labeled_raw, X_val_raw, y_train_labeled, y_val = train_test_split(
		X_labeled_full,
		y_labeled_full,
		test_size=TEST_SIZE,
		stratify=y_labeled_full,
		random_state=SEED
	)

	# --- 1.6 Normalización (StandardScaler) ---
	# Crear y "ajustar" (fit) el scaler SOLO con datos de entrenamiento
	scaler = StandardScaler()
	scaler.fit(X_train_labeled_raw)

	# Aplicar "transform" a TODOS los sets de datos
	X_train_labeled_scaled = scaler.transform(X_train_labeled_raw)
	X_unlabeled_scaled = scaler.transform(X_unlabeled_raw)
	X_val_scaled = scaler.transform(X_val_raw)
	X_test_scaled = scaler.transform(X_test_raw)
	
	# --- 1.7 Reformatear (Reshape) para 1D-CNN ---
	X_train_labeled = X_train_labeled_scaled[:, np.newaxis, :]
	X_unlabeled = X_unlabeled_scaled[:, np.newaxis, :]
	X_val = X_val_scaled[:, np.newaxis, :]
	X_test = X_test_scaled[:, np.newaxis, :]

	y_test = y_test_raw
	print("\n--- Carga de datos completada ---")

	# === 2. Definición de la Búsqueda de Hiperparámetros ===

	# Define los parámetros que quieres probar
	param_grid = {
		'LEARNING_RATE': [1e-3, 5e-4],
		'CONFIDENCE_THRESHOLD': [0.9, 0.95],
		'LAMBDA_U': [1.0, 0.75],
		'FOCAL_GAMMA': [2.0, 3.0]
	}

	grid = ParameterGrid(param_grid)

	all_results = []

	# === 3. Bucle Principal de Búsqueda (Outer Loop) ===

	print(f"\nIniciando búsqueda de parámetros... {len(list(grid))} combinaciones a probar.")

	for run_id, params in enumerate(grid):
		print(f"\n--- [RUN {run_id+1}/{len(list(grid))}] ---")
		print(f"Parámetros: {params}")

		# Extraer parámetros de esta "run"
		CURRENT_LR = params['LEARNING_RATE']
		CURRENT_THRESHOLD = params['CONFIDENCE_THRESHOLD']
		CURRENT_LAMBDA_U = params['LAMBDA_U']
		CURRENT_GAMMA = params['FOCAL_GAMMA']
		
		# Hiperparámetros fijos
		BATCH_SIZE = 32
		EPOCHS = 50 # Puedes bajar esto para pruebas rápidas
		NUM_CLASSES = 5
		
		### CORRECCIÓN: Como pediste, NUM_WORKERS = 0 ###
		NUM_WORKERS = 0 
		PIN_MEMORY = (DEVICE.type == 'cuda')

		# === 4. DataLoaders (Sin cambios) ===
		train_labeled_ds = LabeledECGDataset(X_train_labeled, y_train_labeled)
		train_unlabeled_ds = UnlabeledECGDataset(X_unlabeled)
		val_ds = LabeledECGDataset(X_val, y_val)
		test_ds = LabeledECGDataset(X_test, y_test)

		labeled_loader = DataLoader(
			train_labeled_ds, batch_size=BATCH_SIZE, shuffle=True,
			num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
		)
		unlabeled_loader = DataLoader(
			train_unlabeled_ds, batch_size=BATCH_SIZE, shuffle=True,
			num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
		)
		val_loader = DataLoader(
			val_ds, batch_size=BATCH_SIZE, shuffle=False,
			num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
		)
		test_loader = DataLoader(
			test_ds, batch_size=BATCH_SIZE, shuffle=False,
			num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
		)

		# === 5. Inicializar Modelo, Pérdida y Optimizador ===
		
		model = ECG_1D_CNN(num_classes=NUM_CLASSES).to(DEVICE)
		
		loss_sup = FocalLoss(gamma=CURRENT_GAMMA, alpha=None, reduction='mean').to(DEVICE)
		loss_unsup = FocalLoss(gamma=CURRENT_GAMMA, alpha=None, reduction='none').to(DEVICE)
		
		optimizer = torch.optim.Adam(model.parameters(), lr=CURRENT_LR)

		# === 6. Bucle de Entrenamiento (Inner Loop) ===
		
		best_f1_run = -1.0
		# Guardar el mejor modelo para esta "run" específica
		run_model_path = f"checkpoints/run_{run_id}_best_model.pth" 
		os.makedirs("checkpoints", exist_ok=True)

		for epoch in range(EPOCHS):
			avg_loss_s, avg_loss_u = train_fixmatch(
				model, labeled_loader, unlabeled_loader, optimizer, 
				loss_sup, loss_unsup, DEVICE, CURRENT_THRESHOLD, CURRENT_LAMBDA_U
			)
			
			val_acc, val_f1, val_cm = evaluate(model, val_loader, DEVICE)
			
			if val_f1 > best_f1_run:
				best_f1_run = val_f1
				torch.save(model.state_dict(), run_model_path)
		
		print(f"Run {run_id+1} completada. Mejor F1-Macro de Validación: {best_f1_run:.4f}")
		
		# Guardar los resultados de esta "run"
		all_results.append({
			'run_id': run_id,
			'params': params,
			'best_val_f1': best_f1_run,
			'model_path': run_model_path
		})

	# === 7. Encontrar y Reportar los Mejores Resultados ===

	print("\n--- Búsqueda de Hiperparámetros Finalizada ---")

	# Convertir a DataFrame de Pandas para ver fácil
	results_df = pd.DataFrame(all_results)
	results_df = results_df.sort_values(by='best_val_f1', ascending=False)

	print("Resultados de todas las 'runs':")
	print(results_df)

	# Obtener la mejor "run"
	best_run = results_df.iloc[0]

	print("\n--- 🏆 MEJOR RUN 🏆 ---")
	print(f"Mejor F1-Macro (Validación): {best_run['best_val_f1']:.4f}")
	print(f"Mejores Parámetros: {best_run['params']}")
	print(f"Mejor modelo guardado en: {best_run['model_path']}")

	# === 8. Evaluación Final y CSV con el MEJOR modelo ===

	print("\nCargando el MEJOR modelo para evaluación final en Test...")

	# Cargar el mejor modelo de la mejor "run"
	best_model = ECG_1D_CNN(num_classes=NUM_CLASSES).to(DEVICE)
	best_model.load_state_dict(torch.load(best_run['model_path']))

	test_acc, test_f1, test_cm = evaluate(best_model, test_loader, DEVICE)
	print(f"\n--- Resultados Finales (Test Set) con el Mejor Modelo ---")
	print(f"Accuracy: {test_acc:.4f}")
	print(f"F1-Macro: {test_f1:.4f}")
	print("Matriz de Confusión (Test):")
	print(test_cm)

	# --- 8.1 Generación del Archivo de Submission (Test) ---
	print("\n--- Generando archivo CSV de submission ---")

	# --- 8.2 Cargar los IDs del CSV de test ---
	try:
		test_df_original = pd.read_csv("ecg_signals/test_semi_supervised.csv")
		test_ids = test_df_original.iloc[:, 0].values
		print(f"IDs de test cargados: {len(test_ids)} encontrados.")
	except FileNotFoundError:
		print("Error: No se pudo cargar 'test_semi_supervised.csv' para obtener los IDs.")
		exit()
		
	# --- 8.3 Obtener todas las predicciones del modelo ---
	best_model.eval()
	all_predictions = []

	with torch.no_grad():
		for x_batch, y_true_labels in test_loader:
			x_batch = x_batch.to(DEVICE)
			
			logits = best_model(x_batch)
			
			preds = torch.argmax(logits, dim=1)

			all_predictions.extend(preds.cpu().numpy())

	print(f"Predicciones generadas: {len(all_predictions)} hechas.")

	# --- 8.4 Verificar y Crear el DataFrame ---
	if len(test_ids) != len(all_predictions):
		print(f"¡Error! La cantidad de IDs ({len(test_ids)}) no coincide "
			f"con la cantidad de predicciones ({len(all_predictions)}).")
		print("Verifica que el test_loader NO esté barajando (shuffle=False).")
	else:
		submission_df = pd.DataFrame({
			'ID': test_ids,
			'label': all_predictions
		})

		output_filename = f"ecg_submittions/submission_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv"
		
		submission_df.to_csv(output_filename, index=False)
		
		print(f"\n¡Éxito! Archivo de submission guardado en: {output_filename}")
		print("Vista previa de los resultados:")
		print(submission_df.head())